In [1]:
import os
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace, trim, lower
from pyspark.ml.feature import HashingTF, Tokenizer, StopWordsRemover, IDF
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.conf import SparkConf

grid search
teste com outros algoritmos + parâmetros
- como que várias árvores demora mais do que 1 só?
cross validation
seleção de atributos
randomsplit diferente
numero de pagina no rodapé
mostrar linha do tempo/comparações de resultados
todo o grupo precisa saber explicar

atributos que tem uma correlação muito grande podem as vezes ser eliminados (duração do anime de acordo com tipo de mídia, ous eja, se aparece na tv geralmente tem 22 min)

**(1) DEFINIÇÕES INICIAIS**

In [2]:
# Capturar o dirname do caminho real para este arquivo .ipynb
PATH = os.path.dirname(os.path.realpath('__file__'))
print(PATH)

C:\Users\yagoc\Documents\Linguagem Java\Programação para Ciência de Dados\Projeto_TFIDF


In [3]:
# Definir esquema do dataset
esquema = 'id_documento STRING, assunto INT, titulo STRING, text STRING'

In [4]:
# Definir stopwords
arquivo = open(os.path.join(PATH, 'stop_words_english.txt'), "r", encoding="utf8")

#Criar uma lista com as linhas do arquivo e ignorar quebras de linha ('\n')
list_stopwords = arquivo.read().splitlines()
list_stopwords

['a',
 'aa',
 'able',
 'about',
 'above',
 'abroad',
 'according',
 'accordingly',
 'across',
 'actually',
 'adj',
 'after',
 'afterwards',
 'again',
 'against',
 'ago',
 'ahead',
 "ain't",
 'all',
 'allow',
 'allows',
 'almost',
 'alone',
 'along',
 'alongside',
 'already',
 'also',
 'although',
 'always',
 'am',
 'amid',
 'amidst',
 'among',
 'amongst',
 'an',
 'and',
 'another',
 'any',
 'anybody',
 'anyhow',
 'anyone',
 'anything',
 'anyway',
 'anyways',
 'anywhere',
 'apart',
 'appear',
 'appreciate',
 'appropriate',
 'are',
 "aren't",
 'around',
 'as',
 "a's",
 'aside',
 'ask',
 'asking',
 'associated',
 'at',
 'available',
 'away',
 'awfully',
 'back',
 'backward',
 'backwards',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'becoming',
 'been',
 'before',
 'beforehand',
 'begin',
 'behind',
 'being',
 'believe',
 'below',
 'beside',
 'besides',
 'best',
 'better',
 'between',
 'beyond',
 'both',
 'brief',
 'but',
 'by',
 'came',
 'can',
 'cannot',
 'cant',
 "can't",
 'ca

- **open(os.path.join(PATH, 'stop_words_english.txt'), "r", encoding="utf8")** = Abrir arquivo das stopwords no modo read com codificação utf-8

- UTF-8 (8-bit Unicode Transformation Format) é a codificação de caracteres mais comum da World Wide Web. Cada caractere é representado por um a quatro bytes. UTF-8 é compatível com versões anteriores do ASCII e pode representar qualquer caractere Unicode padrão.
- Unicode é um padrão adotado mundialmente que possibilita com que todos os caracteres de todas as linguagens escritas utilizadas no planeta possam ser representados em computadores.

**(2) SETUP SPARK**

In [5]:
#Configuração das propriedades do Spark
config = SparkConf().setAll([('spark.executor.memory', '27g'), ('spark.executor.cores', '16'), ('spark.cores.max', '16'), ('spark.driver.memory','27g'),
                             ('spark.sql.inMemoryColumnarStorage.compressed', True), ('spark.sql.inMemoryColumnarStorage.batchSize',10000)])

- *https://spark.apache.org/docs/latest/configuration.html*

- **SparkConf.setAll(pairs)** = Defina vários parâmetros, passados ​​como uma lista de pares no estilo (chave, valor).

- **spark.executor.memory** = Quantidade de memória a ser usada por processo do executor, no mesmo formato das strings de memória JVM com um sufixo de unidade de tamanho ("k", "m", "g" ou "t") (por exemplo 512m, 2g).
- **spark.executor.cores** = O número de núcleos a serem usados ​​em cada executor.
- **spark.cores.max** = A quantidade máxima de núcleos de CPU a serem solicitados para o aplicativo de todo o cluster (não de cada máquina).
- **spark.driver.memory** = Quantidade de memória a ser usada para o processo do driver, ou seja, onde SparkContext é inicializado.
- **spark.sql.inMemoryColumnarStorage.compressed** = Quando definido como True, o Spark SQL selecionará automaticamente um codec de compactação para cada coluna com base nas estatísticas dos dados.
- **spark.sql.inMemoryColumnarStorage.batchSize** = Controla o tamanho dos lotes para armazenamento em cache colunar. Tamanhos de lote maiores podem melhorar a utilização e compactação da memória, mas arriscam OOMs ao armazenar dados em cache.

In [6]:
# Definir o Spark Context
sc = SparkContext(master = 'local[2]', appName = 'Clasificação - Assunto News')

- *https://spark.apache.org/docs/latest/api/java/org/apache/spark/SparkContext.html#SparkContext--*
- *https://medium.com/data-hackers/entendo-funcionamento-do-pyspark-2b5ab4321ab9*

- SparkContext representa a conexão a um cluster Spark.
- O construtor alternativo **SparkContext(String master, String appName, SparkConf conf)** nos permite definir propriedades comuns do Spark diretamente.
- O parâmetro master define a URL do cluster ao qual se conectar.
- Só podemos ter um SparkContext ativo por Java Virtual Machine (JVM).

- O Spark Context que é um processo Python que na verdade é apenas usado para comunicação local, este processo inicia uma JVM. Toda comunicação entre nós do clusters, entradas e saídas de arquivos são processadas na JVM.

In [7]:
# Launch User Interface (UI)
sc

<SparkContext master=local[2] appName=Clasificação - Assunto News>

In [8]:
# Criar a Spark Session
spark = SparkSession(sc).builder.getOrCreate()

**(3) ABRINDO E VISUALIZANDO DATASET**

In [9]:
# Lendo dataset
dataset = spark.read.csv(os.path.join(PATH, 'news.csv'), schema = esquema)

- **os.path.join(PATH, 'news.csv')** = Concatena a variável PATH e o nome do dataset

In [10]:
# Verificar valores nulos na coluna
valores_missing = {col: dataset.filter(dataset[col].isNull()).count() for col in dataset.columns}
print(valores_missing)

{'id_documento': 0, 'assunto': 0, 'titulo': 0, 'text': 0}


- **{col: dataset.filter(dataset[col].isNull()).count() for col in dataset.columns}** = Para cada coluna na lista de colunas do dataset, salve em um dicionário no formato (coluna : contagens de valores nulos da coluna)

In [11]:
#Exibir quantidade de notícias por assunto
dataset.groupby('assunto').count().orderBy('assunto').show()

+-------+-----+
|assunto|count|
+-------+-----+
|      1| 1900|
|      2| 1900|
|      3| 1900|
|      4| 1900|
+-------+-----+



In [12]:
#Exibir quantidade de cada id (verificar se existem id's duplicados)
dataset.groupby('id_documento').count().orderBy('count', ascending = False).show()

+------------+-----+
|id_documento|count|
+------------+-----+
|      000118|    1|
|      000543|    1|
|      001247|    1|
|      001281|    1|
|      001375|    1|
|      001416|    1|
|      001438|    1|
|      002275|    1|
|      002276|    1|
|      002616|    1|
|      002668|    1|
|      003141|    1|
|      003363|    1|
|      003518|    1|
|      003623|    1|
|      004184|    1|
|      004525|    1|
|      004586|    1|
|      004683|    1|
|      005061|    1|
+------------+-----+
only showing top 20 rows



**(4) TRANSFORMANDO DADOS**

In [13]:
#Renomear coluna 'assunto' para 'label'
dataset = dataset.withColumnRenamed('assunto', 'label')

In [14]:
# Padronização do Texto
dataset = (dataset.withColumn("text", regexp_replace(col("text"), "\W+", " ")) # \W+ remove caracteres especiais, como: ",@.()-;
                  .withColumn("text", regexp_replace(col("text"), "\d+", " "))  # remove dígitos
                  .withColumn("text", regexp_replace(col("text"), "^[a-z][^a-z]|[^a-z][a-z][^a-z]", " "))  # substitua todas as letras sozinhas e iniciais sozinhas por espaço
                  .withColumn("text", regexp_replace(col("text"), "\\s+", " "))  # remove espaços multiplicados
                  .withColumn("text", trim(col("text")))  # remove espaços em branco no inicio e fim da string
                  .withColumn("text", lower(col("text"))) # transforma todo o texto em minusculo
                  )

In [15]:
# Divisão em treino e teste
#70% para treino e 30% para teste
(dados_treino, dados_teste) = dataset.randomSplit([0.7, 0.3], seed=157)

dados_treino.count()
dados_teste.count()

2306

**(5) MODELAGEM**

In [16]:
# Modelo
lr = LogisticRegression(maxIter = 100, regParam = 0.1)

# Definição dos Stages
tokenizer = Tokenizer(inputCol = 'text', outputCol = "termos")
stopWordsRemover = StopWordsRemover(inputCol=tokenizer.getOutputCol(), outputCol="termos_sem_stopwords").setStopWords(list_stopwords)
hashingTF = HashingTF(inputCol = stopWordsRemover.getOutputCol(), outputCol = 'termos_HashTF')
idf = IDF(inputCol = hashingTF.getOutputCol(), outputCol = 'features')

# Criação do Pipeline
pipeline = Pipeline(stages = [tokenizer, stopWordsRemover, hashingTF, idf, lr])

model = pipeline.fit(dados_treino)

- O estágio **Tokenizer** envolve dividir uma frase em palavras menores. Ele converte o texto de entrada em umo conjunto de dados em tokens de palavras que nossa máquina possa entender.
- O estágio **StopWordsRemover** extrai todas as palavras de parada disponíveis em nosso conjunto de dados. Palavras irrelevantes são um conjunto de palavras usadas com frequência em uma determinada frase. Essas palavras podem ser tendenciosas ao construir o classificador.
- O estágio **TF-IDF** é uma medida estatística que indica a importância de uma palavra em relação a outros documentos em uma coleção de documentos. Isso cria uma relação entre diferentes palavras em um documento. Se uma palavra aparece com frequência em um determinado documento e também aparece com frequência em outros documentos, isso mostra que ela tem pouco poder preditivo para a classificação. Quanto mais a palavra é rara em determinados documentos, mais ela tem valor na análise preditiva.

**(6) AVALIAÇÃO DO MODELO**

In [17]:
avaliador = MulticlassClassificationEvaluator(predictionCol = 'prediction', labelCol = 'label', metricName = 'accuracy')

prediction = model.transform(dados_teste)

# Acurácia do Modelo
print(avaliador.evaluate(prediction))

# Confusion Matrix
confusion_matrix = prediction.groupBy('label', 'prediction').count().orderBy('label', 'prediction')
confusion_matrix.toPandas().to_csv(os.path.join(PATH, 'report_confusion_matrix.csv'), index = False)

0.849956634865568


**(7) SALVAR E ENCERRAR PROCESSOS**

In [18]:
# =============================================================================
# Salvando o Modelo
# =============================================================================
model.write().overwrite().save(os.path.join(PATH, 'model_best'))

In [19]:
# =============================================================================
# Stop User Interface
# =============================================================================
sc.stop()

In [20]:
#Exibir quantidade de cada notícia (verificar se existem notícias duplicadas)
#dataset.groupby('text', 'assunto').count().orderBy('count', ascending = False).show(truncate = True)